In [ ]:
'''
Visualize pressure measurement of BioTac
Author: Yitian Shao
Created on 2022.06.25 based on 'VisualizeBioTac.ipynb'
'''
# %matplotlib notebook 
# %matplotlib notebook 

import time
from os import walk
import os.path as ospa
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import pandas as pd
from scipy import signal
import seaborn as sns

plt.rc('font', size=10, family='Verdana') # 'Tahoma', 'DejaVu Sans', 'Verdana'"
plt.rc('axes', edgecolor='0.5', linewidth=0.75)
plt.rc('axes.spines', **{'bottom':True, 'left':True, 'right':False, 'top':False})
plt.rcParams.update({'errorbar.capsize': 4})

figSize_inch = (3.2, 2.4)

In [ ]:
'''
General Functions
'''

def aPlot(figName='', is3D = False, dpi=72):
    ax = []
    
    fig1 = plt.figure(figsize = (6,3), dpi=dpi)
    
    fig1.suptitle(figName, fontsize=16)
    if(is3D):
        ax = fig1.add_subplot(111, projection='3d')
    else:
        ax = fig1.add_subplot(111)
        
    return ax, fig1

def yyAxisPlot(x0, y0, x1, y1, xText=None, y0Text=None, y1Text=None):
    ax0, fig0 = aPlot(); 
    ax0.set_xlabel(xText)
    ax0.plot(x0, y0, color='tab:orange'); 
    ax0.set_ylabel(y0Text, color='tab:orange')
    ax0.tick_params(axis='y', labelcolor='tab:orange')

    ax1 = ax0.twinx()
    ax1.plot(x1, y1, color='tab:blue');
    ax1.set_ylabel(y1Text, color='tab:blue')
    ax1.tick_params(axis='y', labelcolor='tab:blue')
    fig0.tight_layout() 

def sample2Time(ax, Fs): # Covert the xticks of current plot from samples to time using Fs
    locs, _ = plt.xticks()
    plt.xticks(locs, locs/Fs)
    ax.set_xlim(0, locs[-1])
    ax.set_xlabel('Time (secs)')

def lowpassSmooth(datain, cutFreqRatio = 0.05, order = 8):
    b, a = signal.butter(order, 2 * cutFreqRatio, btype='low')
    dataout = signal.filtfilt(b, a, datain)
    return dataout

def decodeActuatorInfo(rootName):
    actLabel = re.findall('Act\d+mm\d+\.\d+mL', rootName)
    if actLabel:
        actLabel = actLabel[0]
    return actLabel
    
def decodeDataType(fileName):
    dataType = re.findall('Pdc|Pac', fileName)
    if dataType:
        dataType = dataType[0]
    return dataType

def decodeData(fileName, numFormat, frontCode='', rearCode=''):
    segStr = re.findall(frontCode+numFormat+rearCode, fileName)
    if segStr:
        numData = float(re.findall(numFormat, segStr[0])[0])
    else:
        numData = None
    return numData

def cutRepeatTrial(datain, Fs, expectedTrialNum, disp=False):
    rawData = datain[:,1]
    smData = lowpassSmooth(rawData)
    
    smData = signal.detrend(smData, type='linear')
    
    samp = np.arange(len(rawData))
    
    maxRawValue = np.max(smData)
    
    ''' Fine tunning needed for individual measurement session'''
    segPointInd = np.squeeze(np.argwhere(smData[10:] > 0.25 * maxRawValue))+50 # Find value larger than 25% of peak as valid segment data point
    segGapInd = np.squeeze(np.argwhere(np.diff(segPointInd) > 1))# Index of point where large gap occurs (end and start of a seg)
    cutInd = (0.5 * (segPointInd[segGapInd] + segPointInd[segGapInd+1])).astype(int) # Cut in the middle of a end and a start point
    
    if not isinstance(cutInd,np.ndarray) or len(cutInd) != expectedTrialNum-1:
        if disp:
            ax0, _ = aPlot(); 
            ax0.plot(samp, rawData, color='k'); 
            plt.show();       
        return []
    
    avgSegLen = np.mean(np.diff(cutInd))
    cutInd = np.insert(cutInd, 0, max(cutInd[0]-avgSegLen, 0))
    
    if disp:
        ax0, _ = aPlot(); 
        ax0.plot(samp, rawData, color='k'); 
#         ax0.plot(samp, smData, color='tab:blue');
        ax0.plot(cutInd, np.zeros(cutInd.shape), '*r')
        plt.show();
    
    return cutInd

def selectConditions(dFrame, condiList):
    indList = []
    for aCondi in condiList:
        indList.extend(dFrame[dFrame['Label'] == aCondi].index)

    return indList

def getDataArray(dFrame, indList, colName, toFlat=False):
    compValue = (dFrame.loc[indList,colName]).to_numpy()  
    
    if toFlat:
        flatValue = []
        for aRow in compValue:
            flatValue.extend(aRow)
        flatValue = np.array(flatValue)
        return flatValue
            
    return compValue
    
def getMeanSTD(compValue): # Compute errorbar of data mixing all conditions (rows of input array)
    if len(compValue) == 0:
        return None, None
        
    compValue = np.array(compValue)
    
    meanValue = np.mean(compValue)
    stdValue = np.std(compValue)
    
    return meanValue, stdValue

In [ ]:
'''
Load, preprocess and segment BioTac data

Ref: https://syntouchinc.com/wp-content/uploads/2020/09/SynTouch-Product-Manual-BioTac-2020-09-23.pdf
'''

biotacFs = {
  "Pac": 2200,
  "Pdc": 100,
} # The sampling frequency of BioTac depends on data type. 

biotacUnit = {
  "Pac": 0.00037,
  "Pdc": 0.0365,
} # The unit conversion of BioTac depends on data type. Pac: 0.00037kPa/bit, Pdc:  0.0365kPa/bit

measureDataPath = ".\Data2022.06.26"

DCpressure = []
ACpressure = []

''' Data preprocess and segmentation '''
for root, directories, files in walk(measureDataPath):
    for fileName in files:
        actLabel = decodeActuatorInfo(root)
        
        if actLabel:
            tubeLen = decodeData(actLabel, '\d+', rearCode='mm')
            infillVol = decodeData(actLabel, '[\d+\.]*\d+', rearCode='mL')   
        
        dataType = decodeDataType(fileName)  
        
        if dataType:
            vLevel = decodeData(fileName, '\d+', frontCode='v')
            cTime = decodeData(fileName, '\d+', frontCode='c')
            dTime = decodeData(fileName, '\d+', frontCode='d')
            trialNum = decodeData(fileName, '\d+', frontCode='t')
        
#             print("Len=%d, Infill=%.2f, Condi: %s v=%d c=%d d=%d t=%d" % 
#                   (tubeLen, infillVol, dataType, vLevel, cTime, dTime, trialNum))         
            dLabel = "L%03dF%.1fV%03dC%04dD%04d" % (tubeLen, infillVol, vLevel, cTime, dTime)
    
            ''' Read in data '''
            data = np.genfromtxt(ospa.join(root, fileName), delimiter=',')
            Fs = biotacFs[dataType]
            
            inactiveValue = np.mean(np.concatenate((data[:int(0.2*Fs),1], data[-int(2*Fs):,1])))
            
            ''' Unit conversion process suggested by BioTac manual '''
            data[:,1] = data[:,1] - inactiveValue # Remove DC offset
            data[:,1] = data[:,1] * biotacUnit[dataType] # Convert from bit to physical unit (kPa)
            
            if dataType == 'Pdc':
                cutInd = cutRepeatTrial(data, Fs, expectedTrialNum=6, disp=False)

                DCpressure.append([dLabel, tubeLen, infillVol, dataType, vLevel, cTime, dTime, 
                                   trialNum, data, cutInd])
            elif dataType == 'Pac':
                ACpressure.append([dLabel, tubeLen, infillVol, dataType, vLevel, cTime, dTime, 
                                   trialNum, data])
                
DCpressure = pd.DataFrame(DCpressure, columns = ['Label', 'TubeLength_mm','Infill_mL','DType',
                 'Voltage','ChargeTime_ms','DischargeTime_ms','Trial','Data','CutIndex'])  
ACpressure = pd.DataFrame(ACpressure, columns = ['Label', 'TubeLength_mm','Infill_mL','DType',
                 'Voltage','ChargeTime_ms','DischargeTime_ms','Trial','Data']) 

DCpressure.head(10);

In [ ]:
'''
Data restructure (grouping segmented measurements) for analysis and visualization
'''

dictData = {} # Use dictionary to avoid dataframe concatenation errors

for i in DCpressure.index:
    dLabel = DCpressure.loc[i, 'Label']
    dcData = DCpressure.loc[i, 'Data']
    cutIndDC = DCpressure.loc[i, 'CutIndex']
    
    ind = ((ACpressure['Label'] == DCpressure.loc[i, 'Label']) & 
        (ACpressure['Trial'] == DCpressure.loc[i, 'Trial']))   
    acData = ACpressure.loc[ind, 'Data'].values[0]
    
    segNum = len(cutIndDC)
    if segNum: # When data can be segmented
        cutIndAC = cutIndDC * int(biotacFs['Pac'] / biotacFs['Pdc']) # Adjust cut index considering difference in sampling frequency
    
        if dLabel not in dictData: # Initialization for a new label
            dictData[dLabel] = DCpressure.loc[i, ['Label', 'TubeLength_mm','Infill_mL',
                     'Voltage','ChargeTime_ms','DischargeTime_ms']]
        
        if 'Pdc' not in dictData[dLabel]: # Initialization for a new column of current label
            dictData[dLabel]['Pdc'] = []
            
        if 'Pac' not in dictData[dLabel]: # Initialization for a new column of current label
            dictData[dLabel]['Pac'] = []
        
        for j in range(segNum):
            if j < segNum-1:
                segDC = dcData[cutIndDC[j]:cutIndDC[j+1],:]
                segAC = acData[cutIndAC[j]:cutIndAC[j+1],:]
            else:
                segDC = dcData[cutIndDC[j]:,:]
                segAC = acData[cutIndAC[j]:,:]

            dictData[dLabel]['Pdc'].append(segDC)  
            dictData[dLabel]['Pac'].append(segAC) 
            
#             yyAxisPlot(segDC[:,0], segDC[:,1], segAC[:,0], segAC[:,1], 
#                        xText=("%s  Time (s)" % dLabel), 
#                        y0Text='DC pressure (kPa)', y1Text='AC pressure (kPa)')
#             plt.show();
    else:
        print("[Warining] Data cannot be segmented: %s" % dLabel)
        
        
''' Convert the disctionary to a dataframe '''
allData = None

for dLabel in dictData:
    aFrame = dictData[dLabel].to_frame().T   
    
    if allData is None:
        allData = aFrame
    else:
        allData = pd.concat([allData, aFrame], axis=0)
    
display(allData)    

In [ ]:
'''Feature extraction from signals: peak value, time-averaged value, rising time'''

column_peakDC = []
column_tavgDC = []
column_triseDC = []

for _, aRow in allData.iterrows():
    print(aRow['Label'])
#     print(\"Len=%d, Infill=%.2f, Condi: v=%d c=%d d=%d\" % (aRow['TubeLength_mm'], aRow['Infill_mL'], aRow['Voltage'],
#                    aRow['ChargeTime_ms'], aRow['DischargeTime_ms'])) 
    peakDC = []
    tavgDC = []
    triseDC = []
    
    for aSignal in aRow['Pdc']:
        smData = lowpassSmooth(aSignal[:,1])
        smData = signal.detrend(smData, type='linear')
   
        maxValue = np.max(smData)
        onsetInd = np.argwhere(smData[50:] > 0.20 * maxValue)[0][0]+50 # When signal reach 10% of its peak (onset)
        
        PP90Ind = np.argwhere(smData[50:] > 0.80 * maxValue)[0][0]+50 # When signal reach 90% of its peak
       
        peakInd = np.argmax(smData) #################### To be investigated ?smData/aSignal[:,1]
        peakValue = np.max(aSignal[:,1])
  
        riseTime = aSignal[PP90Ind,0] - aSignal[onsetInd,0]
        tavgValue = np.mean(aSignal[onsetInd:peakInd,1])
        
        peakDC.append(peakValue)
        tavgDC.append(tavgValue)
        triseDC.append(riseTime)
      
#         ax1,_ = aPlot()
#         ax1.plot(aSignal[:,0], aSignal[:,1], 'k')
#         ax1.plot(aSignal[:,0], smData, 'g')
#         ax1.plot([aSignal[onsetInd,0], aSignal[onsetInd,0]], [0, maxValue], '-r')
#         ax1.plot([aSignal[PP90Ind,0], aSignal[PP90Ind,0]], [0, maxValue], '-r')
#         ax1.set_xlabel("peak=%.1fkPa - tAvg=%.1fkPa - riseT=%.3fs" % (peakValue,tavgValue,riseTime))
#         plt.show()
    
    column_peakDC.append(peakDC)
    column_tavgDC.append(tavgDC)
    column_triseDC.append(triseDC)
    
allData['PeakPdc'] = column_peakDC
allData['AvgPdc'] = column_tavgDC
allData['TrisePdc'] = column_triseDC

display(allData)

In [ ]:
# ''' 
# Signal check (optional, may be used for future when crosscorrelation of trials is desired)
# '''

for _, aRow in allData.iterrows():
    print(aRow['Label'])
    print("Len=%d, Infill=%.2f, Condi: v=%d c=%d d=%d" % 
                  (aRow['TubeLength_mm'], aRow['Infill_mL'], aRow['Voltage'], 
                   aRow['ChargeTime_ms'], aRow['DischargeTime_ms']))     
    
    ax0,_ = aPlot()
    for aSignal in aRow['Pdc']:
        t = aSignal[:,0] - aSignal[0,0]
        ax0.plot(t, aSignal[:,1])
    ax0.set_ylabel("DC pressure (kPa)")
    
    ax1,_ = aPlot()
    for aSignal in aRow['Pac']:
        t = aSignal[:,0] - aSignal[0,0]
        ax1.plot(t, aSignal[:,1])
    ax1.set_ylabel("AC pressure (kPa)")
    
    plt.show();


In [ ]:
'''
Visualize the 42 grid: vLevel = [2, 4, 8, 16, 32, 50, 100] * cTime = [100, 200, 400, 800, 1200, 1600]
'''
def computeErrorbar(dframe, xColName, xColList, yColName, yColList, zColName):
    dMean = []
    dSTD = []
    
    for x in xColList:
        dMean.append([])
        dSTD.append([])
        for y in yColList:
            selectInd = (dframe[xColName] == x) & (dframe[yColName] == y)

            compValue = getDataArray(dframe, selectInd, zColName, toFlat=True)  
            
            valueMean, valueSTD = getMeanSTD(compValue)
            
            dMean[-1].append(valueMean)
            dSTD[-1].append(valueSTD)
    
    dMean = np.array(dMean)
    dSTD = np.array(dSTD)
    
    return dMean, dSTD
 
def dispErrorbar(ax, x, dMean, dErr, xLabel, xLabelStr, yLabelStr, zLabel, colorPalette=None):
    yLen = len(yLabelStr)
    
#     colors = cm.rainbow(np.linspace(0, 1, yLen))
    colors = sns.color_palette(colorPalette, n_colors=yLen)
    
    ax.set_ylabel(zLabel, color='k')
    ax.tick_params(axis='y', labelcolor='k')
    ax.spines['bottom'].set_color('k')
    ax.spines['top'].set_color('k') 
    ax.spines['right'].set_color('k')
    ax.spines['left'].set_color('k')

#     ax.violinplot(dMean, positions=x, showmeans=False, showmedians=False, showextrema=False)
    pltHandles = []
    for i in np.arange(yLen):
        pltHandles.append(ax.plot(2*x + (i * 0.1), dMean[i], '.-', c=colors[i])[0])
        (ax.errorbar(2*x + (i * 0.1), dMean[i], yerr=dErr[i], fmt='none', ecolor=colors[i], capsize=0)[0]) #ecolor='tab:grey'
        
    ax.set_xlabel(xLabel)
    ax.set_xticklabels(xLabelStr, rotation=0);
    ax.set_xticks(2*x + 0.4);

    ax.legend(handles=reversed(pltHandles), labels=reversed(yLabelStr), frameon=False, bbox_to_anchor=(1.02, 1.0), 
              loc='upper left');
    
    return ax

In [ ]:
''' Evaluation and visualization with designated features '''
vLevels = [2, 4, 8, 16, 32, 64, 100]
cTime = [100, 200, 400, 800, 1200, 1600]

x = np.arange(len(cTime))
xLabelStr = np.array(cTime) / 1000
yLabelStr = ["%d%%" % y for y in vLevels]
xLabel = 'Charge Time (sec)'

meanPeakPdc, stdPeakPdc = computeErrorbar(allData, 'Voltage', vLevels, 'ChargeTime_ms', cTime, 'PeakPdc')   
meanAvgPdc, stdAvgPdc = computeErrorbar(allData, 'Voltage', vLevels, 'ChargeTime_ms', cTime, 'AvgPdc')    
meanTrisePdc, stdTrisePdc = computeErrorbar(allData, 'Voltage', vLevels, 'ChargeTime_ms', cTime, 'TrisePdc')

meanPeakPdc[:2, 0] = np.NaN
meanAvgPdc[:2, 0] = np.NaN
meanTrisePdc[:2, 0] = np.NaN
stdPeakPdc[:2, 0] = np.NaN
stdAvgPdc[:2, 0] = np.NaN
stdTrisePdc[:2, 0] = np.NaN

fig1, ax1 = plt.subplots(dpi=300, figsize=figSize_inch)
dispErrorbar(ax1, x, meanPeakPdc, stdPeakPdc, xLabel, xLabelStr, yLabelStr, 'Peak Pressure (kPa)', 'Set2')

fig2, ax2 = plt.subplots(dpi=300, figsize=figSize_inch)
dispErrorbar(ax2, x, meanAvgPdc, stdAvgPdc, xLabel, xLabelStr, yLabelStr, 'Time-avg. Pressure (kPa)', 'Set2')

fig3, ax3 = plt.subplots(dpi=300, figsize=figSize_inch)
dispErrorbar(ax3, x, meanTrisePdc, stdTrisePdc, xLabel, xLabelStr, yLabelStr, 'Rise time (sec)', 'Set2')

# fig1.savefig("mixData_Peak.pdf", bbox_inches='tight')
# fig2.savefig("mixData_Tavg.pdf", bbox_inches='tight')
# fig3.savefig("mixData_Trise.pdf", bbox_inches='tight')

In [ ]:
'''
Statistical test of voltage level distinctiveness
'''
import scipy.stats as stats

def getflatData(dframe, independVarName, iVar, condiColName, condi, zColName):
    selectInd = (dframe[independVarName] == iVar) & (dframe[condiColName] == condi)
    
    compValue = getDataArray(dframe, selectInd, zColName, toFlat=True)  
        
    return compValue

def compANOVA(dframe, independVarName, independVarList, condiColName, condiColList, zColName):
    pThresh = 0.05
    
    print("If sample size is sufficiently large and group mean is the preferred measure of central tendency, parametric tests are the way to go.")
    
    for condi in condiColList:
        iVarList = independVarList.copy()
        if condi == 100:
            print("Exclude noisy data")
            iVarList = iVarList[2:]

        '''1-way ANOVA'''
        f, p = stats.f_oneway(*[list(getflatData(allData, independVarName, iVar, condiColName, condi, zColName)) 
                                for iVar in iVarList])
        outString = ("\nANOVA: %s = %d - independent var = %s, dependent var = %s, F = %.2f, p = %.6f" % 
              (condiColName, condi, independVarName, zColName, f, p))

        print("\x1b[31m\"%s\"\x1b[0m" % outString) if p > pThresh else print(outString)
        
        '''t test and Mann-Whitney U Test (two group)'''
        print("(two sample groups) t-Test [T]")
        for i in range(len(iVarList)-1):
            print("Compare %s: %d vs %d" % (independVarName, iVarList[i], iVarList[i+1]))
            
            stat, p = stats.ttest_ind(*[list(getflatData(allData, independVarName, iVar, condiColName, condi, zColName)) 
                                    for iVar in iVarList[i:i+2]])
            outString = ("[T] %s = %d - independent var = %s, dependent var = %s, Statistics = %.2f, p = %.6f" % 
                  (condiColName, condi, independVarName, zColName, f, p))  
            print("\x1b[31m\"%s\"\x1b[0m" % outString) if p > pThresh else print(outString)
        
            stat, p = stats.mannwhitneyu(*[list(getflatData(allData, independVarName, iVar, condiColName, condi, zColName)) 
                                    for iVar in iVarList[i:i+2]])
            outString = ("[M] %s = %d - independent var = %s, dependent var = %s, Statistics = %.2f, p = %.6f" % 
                  (condiColName, condi, independVarName, zColName, f, p))
            print("\x1b[31m\"%s\"\x1b[0m" % outString) if p > pThresh else print(outString)
        
#         '''Kruskal-Wallis H-test'''
#         stat, p = stats.kruskal(*[list(getflatData(allData, independVarName, iVar, condiColName, condi, zColName)) 
#                                 for iVar in iVarList])
#         outString = ("Kruskal-Wallis H-test: %s = %d - independent var = %s, dependent var = %s, \nStatistics = %.2f, p = %.6f" % 
#               (condiColName, condi, independVarName, zColName, f, p))
#         print("\x1b[31m\"%s\"\x1b[0m" % outString) if p > pThresh else print(outString)

compANOVA(allData, 'Voltage', vLevels, 'ChargeTime_ms', cTime, 'PeakPdc')   

compANOVA(allData, 'Voltage', vLevels, 'ChargeTime_ms', cTime, 'TrisePdc')  


In [ ]:
''' Compare biotac measurement against the energy model prediction - cTime 0.4sec'''
ctIndex = 2

vLevels = [2, 4, 8, 16, 32, 64, 100]
cTime = [100, 200, 400, 800, 1200, 1600]

simData = pd.read_csv("./data/EnModelResult_O_cT0.4sec.csv") # Data from model simulation
simPFull = np.array(simData['PFull_Pa'].values, dtype=np.float64)/1000
print(simData['CondiName']);print(simPFull);

x = np.arange(len(vLevels))
xLabelStr = np.array(vLevels)
xLabel = 'Voltage Level (%)'

meanAvgPdcCT, stdAvgPdcCT = computeErrorbar(allData, 'ChargeTime_ms', cTime, 'Voltage', vLevels, 'AvgPdc') 


fig0, ax1 = plt.subplots(dpi=300, figsize=figSize_inch)
fig0.suptitle('cTime 0.4sec - Osc. Measure Volt vs Model predict.', fontsize=6)
ax1.set_xlabel('PWM (%)')
ax1.set_xticklabels(xLabelStr, rotation=0);
ax1.set_xticks(x)
    
ax1.plot(x, meanAvgPdcCT[ctIndex], '.-', c='tab:orange')
ax1.errorbar(x, meanAvgPdcCT[ctIndex], yerr=stdAvgPdcCT[ctIndex], fmt='none', ecolor='tab:orange', capsize=0)
ax1.set_ylabel('Biotac t-avg. Pressure (kPa)', color='tab:orange')
ax1.tick_params(axis='y', labelcolor='tab:orange')

ax2 = ax1.twinx() 
ax2.plot(x, simPFull, '--', color='tab:blue')
ax2.set_ylabel('P Upper Bound (kPa)', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')
ax2.set_ylim([0, 20]);

In [ ]:
''' Compare biotac measurement against the energy model prediction - cTime 1.2sec'''
ctIndex = 4

simData = pd.read_csv("./data/EnModelResult_O_cT1.2sec.csv") # Data from model simulation
simPFull = np.array(simData['PFull_Pa'].values, dtype=np.float64)/1000
print(simPFull);

fig0, ax1 = plt.subplots(dpi=300, figsize=figSize_inch)
fig0.suptitle('cTime 1.2sec - Osc. Measure Volt vs Model predict.', fontsize=6)
ax1.set_xlabel('PWM (%)')
ax1.set_xticklabels(xLabelStr, rotation=0);
ax1.set_xticks(x)
ax1.plot(x, meanAvgPdcCT[ctIndex], '.-', c='tab:orange')
ax1.errorbar(x, meanAvgPdcCT[ctIndex], yerr=stdAvgPdcCT[ctIndex], fmt='none', ecolor='tab:orange', capsize=0)
ax1.set_ylabel('Biotac t-avg. Pressure (kPa)', color='tab:orange')
ax1.tick_params(axis='y', labelcolor='tab:orange')

ax2 = ax1.twinx() 
ax2.plot(x, simPFull, '--', color='tab:blue')
ax2.set_ylabel('P Upper Bound (kPa)', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')
ax2.set_ylim([0, 15]);

In [ ]:
''' Compare biotac measurement against the energy model prediction - cTime 0.2sec'''
ctIndex = 1

simData = pd.read_csv("./data/EnModelResult_O_cT0.2sec.csv") # Data from model simulation
simPFull = np.array(simData['PFull_Pa'].values, dtype=np.float64)/1000
print(simPFull);

fig0, ax1 = plt.subplots(dpi=300, figsize=figSize_inch)
fig0.suptitle('cTime 0.2sec - Osc. Measure Volt vs Model predict.', fontsize=6)
ax1.set_xlabel('PWM (%)')
ax1.set_xticklabels(xLabelStr, rotation=0);
ax1.set_xticks(x)
ax1.plot(x, meanAvgPdcCT[ctIndex], '.-', c='tab:orange')
ax1.errorbar(x, meanAvgPdcCT[ctIndex], yerr=stdAvgPdcCT[ctIndex], fmt='none', ecolor='tab:orange', capsize=0)
ax1.set_ylabel('Biotac t-avg. Pressure (kPa)', color='tab:orange')
ax1.tick_params(axis='y', labelcolor='tab:orange')

ax2 = ax1.twinx() 
ax2.plot(x, simPFull, '--', color='tab:blue')
ax2.set_ylabel('P Upper Bound (kPa)', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')
ax2.set_ylim([0, 25]);

In [ ]:
''' Compare biotac measurement against the energy model prediction - cTime 1.0sec'''
vLevels = [2, 4, 8, 16, 32, 64, 100]
cTime = [100, 200, 400, 800, 1200, 1600]

simData = pd.read_csv("./data/EnModelResult_O_cT1.0sec.csv") # Data from model simulation
simPFull = np.array(simData['PFull_Pa'].values, dtype=np.float64)/1000
print(simData['CondiName']);print(simPFull);

x = np.arange(len(vLevels))
xLabelStr = np.array(vLevels)
xLabel = 'Voltage Level (%)'

meanAvgPdcCT, stdAvgPdcCT = computeErrorbar(allData, 'ChargeTime_ms', cTime, 'Voltage', vLevels, 'AvgPdc') 


fig0, ax1 = plt.subplots(dpi=300, figsize=figSize_inch)
fig0.suptitle('cTime 1.0sec - Osc. Measure Volt vs Model predict.', fontsize=6)
ax1.set_xlabel('R (mm)')
ax1.plot(x, meanAvgPdcCT[2], '.-', c='tab:orange')
ax1.errorbar(x, meanAvgPdcCT[2], yerr=stdAvgPdcCT[2], fmt='none', ecolor='tab:orange', capsize=0)
ax1.set_ylabel('Biotac t-avg. Pressure (kPa)', color='tab:orange')
ax1.tick_params(axis='y', labelcolor='tab:orange')

ax2 = ax1.twinx() 
ax2.plot(x, simPFull, '--', color='tab:blue')
ax2.set_ylabel('P Upper Bound (kPa)', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')

In [ ]:
''' Compare biotac measurement against the energy model prediction with energy loss '''
vLevels = [2, 4, 8, 16, 32, 64, 100]
cTime = [100, 200, 400, 800, 1200, 1600]

simData = pd.read_csv("./data/EnModelResult_amp.csv") # Data from model simulation
simPFull = np.array(simData['PFull_Pa'].values, dtype=np.float64)/1000
print(simData['CondiName']);print(simPFull);

# ''' pLoss compensation '''
# simData['p1SecLoss'] = simData['lossFactor']*simData['dVFull_m3']/1000   # divide the simData['deltaT'] in later part
# simData.loc[0,'p1SecLoss'] = 0
# pLoss = np.tile(np.array(simData.loc[1:,'p1SecLoss'].values, dtype=np.float64), (len(cTime),1)).T 
# pLoss = pLoss / meanTrisePdc # ''' pLoss compensation '''
# pLoss[:2, 0] = 0 # ''' pLoss compensation '''

x = np.arange(len(vLevels))
xLabelStr = np.array(vLevels)
yLabelStr = ["%.1fs" % (y/1000) for y in cTime] 
xLabel = 'Voltage Level (%)'

meanAvgPdcCT, stdAvgPdcCT = computeErrorbar(allData, 'ChargeTime_ms', cTime, 'Voltage', vLevels, 'AvgPdc') 

meanAvgPdcCT[0,:2] = np.NaN
stdAvgPdcCT[0,:2] = np.NaN

''' Amplitdue data - inaccurate '''
fig4, ax4 = plt.subplots(dpi=300, figsize=figSize_inch)
dispErrorbar(ax4, x, meanAvgPdcCT, stdAvgPdcCT, xLabel, xLabelStr, yLabelStr, 'Time-avg. Pressure (kPa)', 'Set2')

ax5 = ax4.twinx() 
ax5.plot(2*x + 0.4, simPFull[1:], '-', color='tab:blue')
ax5.tick_params(axis='y', labelcolor='tab:blue', size=1)


# ''' pLoss compensation '''
# colors = sns.color_palette('Set2', n_colors=len(cTime))
# for i in range(len(cTime)):
#     if i == 0:
#         simPFull[1:3] = 0
#     ax4.plot(2*x + 0.4, simPFull[1:] - pLoss[:,i], '-', color=colors[i])

    
# _, ax5 = plt.subplots(dpi=300, figsize=figSize_inch)
# ax5.plot(2*x + 0.4, simPFull[1:], '-', color='k')
# ax5.set_xlabel(xLabel)
# ax5.set_xticklabels(xLabelStr, rotation=0);
# ax5.set_xticks(2*x + 0.4);